In [1]:
import numpy as np
import pandas as pd

import itertools

import torch
import torch.nn as nn
import torch.optim as optim

from hypll import nn as hnn
from hypll.tensors import TangentTensor
from hypll.optim import RiemannianAdam
from hypll.manifolds.poincare_ball import Curvature, PoincareBall

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [2]:
import sys
sys.path.append('../')
import util

In [3]:
df_train_X = pd.read_csv('../data/beer_features_train_samples.csv', index_col=0)
df_train_y = pd.read_csv('../data/beer_labels_panel_train_samples.csv', index_col=0)
df_val_X = pd.read_csv('../data/beer_features_train.csv', index_col=0)
df_val_y = pd.read_csv('../data/beer_labels_panel_train.csv', index_col=0)
df_test_X = pd.read_csv('../data/beer_features_test.csv', index_col=0)
df_test_y = pd.read_csv('../data/beer_labels_panel_test.csv', index_col=0)

train_X = df_train_X.values
train_y = df_train_y.values
val_X = df_val_X.values
val_y = df_val_y.values
test_X = df_test_X.values
test_y = df_test_y.values

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape

((175000, 231), (175000, 50), (175, 231), (175, 50), (75, 231), (75, 50))

In [5]:
FOLDS = 5
NUM_SAMPLE_TYPES = len(val_X)
NUM_SAMPLES_PER_TYPE = len(train_X) // NUM_SAMPLE_TYPES

fold_nums = list(range(FOLDS))
[num*NUM_SAMPLE_TYPES for num in fold_nums]
[(num+1)*NUM_SAMPLE_TYPES for num in fold_nums]

FOLD_INDICIES = util.get_fold_indices(NUM_SAMPLE_TYPES, FOLDS)

# FOLD_INDICIES = list(zip([num*NUM_SAMPLE_TYPES//FOLDS for num in fold_nums],
#                          [(num+1)*NUM_SAMPLE_TYPES//FOLDS for num in fold_nums]))

print(FOLD_INDICIES)

[(0, 35), (35, 70), (70, 105), (105, 140), (140, 175)]


In [8]:
# Define custom PyTorch dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

<h1> Hyperbolic </h1>

In [9]:
# Define your MLP model
class HYP_MLP(nn.Module):
    def __init__(self, input_size, output_size, layer_size, num_hidden_layers, manifold):
        super(HYP_MLP, self).__init__()
        torch.manual_seed(42)
        self.fc_in = hnn.HLinear(input_size, layer_size, manifold=manifold)
        self.relu = hnn.HReLU(manifold=manifold)
        self.hidden_fcs = nn.ModuleList([hnn.HLinear(layer_size, layer_size, manifold=manifold) for _ in range(num_hidden_layers)])
        self.fc_out = hnn.HLinear(layer_size, output_size, manifold=manifold)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.relu(x)
        for fc in self.hidden_fcs:
            x = fc(x)
            x = self.relu(x)
        x = self.fc_out(x)

        return x


# Define training function
def hyp_train_model(model, train_loader, criterion, optimizer, manifold, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        tangents = TangentTensor(data=inputs, man_dim=-1, manifold=manifold)
        manifold_inputs = manifold.expmap(tangents)

        outputs = model(manifold_inputs)

        loss = criterion(outputs.tensor, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

<h1> EUCLIDEAN </h1>

In [10]:
# Define your MLP model
class EUC_MLP(nn.Module):
    def __init__(self, input_size, output_size, layer_size, num_hidden_layers):
        super(EUC_MLP, self).__init__()
        torch.manual_seed(42)
        self.fc_in = nn.Linear(input_size, layer_size)
        self.relu = nn.ReLU()
        self.hidden_fcs = nn.ModuleList([nn.Linear(layer_size, layer_size) for _ in range(num_hidden_layers)])
        self.fc_out = nn.Linear(layer_size, output_size)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.relu(x)
        for fc in self.hidden_fcs:
            x = fc(x)
            x = self.relu(x)
        x = self.fc_out(x)

        return x

# Define training function
def euc_train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

In [11]:
# param_grid = {
#     'model_type': ['hyp', 'euc'],
#     'num_hidden_layers': [0,1,2,4,8],
#     'layer_size': [64,128,256],
#     'lr': [0.001,0.003,0.01],
#     'weight_decay': [0.001],
#     'batch_size': [1024],
#     'epochs': [10],
#     'curvature': [-1]
# }
param_grid = {
    'model_type': ['euc'],
    'num_hidden_layers': [0],
    'layer_size': [256],
    'lr': [0.003],
    'weight_decay': [0.001],
    'batch_size': [1024],
    'epochs': [10],
    'curvature': [-1]
}

param_combinations = list(itertools.product(*param_grid.values()))
len(param_combinations)

1

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

param_eval_stats = []

for i, params in enumerate(param_combinations):
    print(f'----- Combination {i} -----')
    print(*zip(param_grid.keys(), params))
    model_type, num_hidden_layers, layer_size, lr, weight_decay, batch_size, epochs, curvature = params

    for fold, (fold_start, fold_stop) in enumerate(FOLD_INDICIES):
        print(f'Fold {fold}')

        fold_train_X = train_X[[*list(range(fold_start*NUM_SAMPLES_PER_TYPE)), *list(range(fold_stop*NUM_SAMPLES_PER_TYPE, len(train_X)))]]
        fold_train_y   =   train_y[[*list(range(fold_start*NUM_SAMPLES_PER_TYPE)), *list(range(fold_stop*NUM_SAMPLES_PER_TYPE, len(train_X)))]]
        fold_val_X   = val_X[list(range(fold_start, fold_stop))]
        fold_val_y     =   val_y[list(range(fold_start, fold_stop))]

        train_dataset = CustomDataset(fold_train_X, fold_train_y)
        val_dataset = CustomDataset(fold_val_X, fold_val_y)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        if model_type == 'hyp':
            manifold = PoincareBall(c=Curvature(curvature))
        elif model_type == 'euc':
            manifold = None

        if model_type == 'hyp':
            model = HYP_MLP(input_size=train_X.shape[1],
                            output_size=train_y.shape[1],
                            layer_size=layer_size,
                            num_hidden_layers=num_hidden_layers,
                            manifold=manifold).to(device)
        elif model_type == 'euc':
            model = EUC_MLP(input_size=train_X.shape[1],
                            output_size=train_y.shape[1],
                            layer_size=layer_size,
                            num_hidden_layers=num_hidden_layers).to(device)

        criterion = nn.MSELoss()

        if model_type == 'hyp':
            optimizer = RiemannianAdam(model.parameters(), lr=lr, weight_decay=weight_decay)
        elif model_type == 'euc':
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        eval_stats = {'loss': {'train': [], 'val': []}, 'mae': {'train': [], 'val': []}}

        for epoch in range(epochs):
            if model_type == 'hyp':
                eval_stats['loss']['train'].append(hyp_train_model(model, train_loader, criterion, optimizer, manifold, device))
                eval_stats['loss']['val'].append(util.h_evaluate_loss(model, val_loader, criterion, manifold, device))

                eval_stats['mae']['train'].append(util.h_evaluate_r2(model, train_loader, manifold, device))
                eval_stats['mae']['val'].append(util.h_evaluate_r2(model, val_loader, manifold, device))
            elif model_type == 'euc':
                eval_stats['loss']['train'].append(euc_train_model(model, train_loader, criterion, optimizer, device))
                eval_stats['loss']['val'].append(util.evaluate_loss(model, val_loader, criterion, device))

                eval_stats['mae']['train'].append(util.evaluate_r2(model, train_loader, device))
                eval_stats['mae']['val'].append(util.evaluate_r2(model, val_loader, device))

        print(eval_stats['mae']['val'])
        param_eval_stats.append(eval_stats)

----- Combination 0 -----
('model_type', 'euc') ('num_hidden_layers', 0) ('layer_size', 256) ('lr', 0.003) ('weight_decay', 0.001) ('batch_size', 1024) ('epochs', 10) ('curvature', -1)
Fold 0
(140000, 231)
(140000, 50)
(35, 231)
(35, 50)


KeyboardInterrupt: 

<h1>BEST MODEL</h1>

In [20]:
best_params = {
    'model_type': 'hyp',
    'num_hidden_layers': 0,
    'layer_size': 256,
    'lr': 0.003,
    'weight_decay': 0.001,
    'batch_size': 1024,
    'epochs': 10,
    'curvature': -1
}

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_type, num_hidden_layers, layer_size, lr, weight_decay, batch_size, epochs, curvature = best_params.values()

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(train_X, train_y)):
    print(f'Fold {fold}')
    # fold_train_X, fold_val_X = train_X[train_idx], train_X[val_idx]
    # fold_train_y, fold_val_y = train_y[train_idx], train_y[val_idx]
    fold_train_X, fold_val_X = train_X[train_idx], test_X
    fold_train_y, fold_val_y = train_y[train_idx], test_y
    # fold_train_X, fold_val_X = train_X, test_X
    # fold_train_y, fold_val_y = train_y, test_y

    train_dataset = CustomDataset(fold_train_X, fold_train_y)
    val_dataset = CustomDataset(fold_val_X, fold_val_y)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    if model_type == 'hyp':
        manifold = PoincareBall(c=Curvature(curvature))
    elif model_type == 'euc':
        manifold = None

    if model_type == 'hyp':
        model = HYP_MLP(input_size=train_X.shape[1],
                        output_size=train_y.shape[1],
                        layer_size=layer_size,
                        num_hidden_layers=num_hidden_layers,
                        manifold=manifold).to(device)
    elif model_type == 'euc':
        model = EUC_MLP(input_size=train_X.shape[1],
                        output_size=train_y.shape[1],
                        layer_size=layer_size,
                        num_hidden_layers=num_hidden_layers).to(device)

    criterion = nn.MSELoss()

    if model_type == 'hyp':
        optimizer = RiemannianAdam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif model_type == 'euc':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    eval_stats = {'loss': {'train': [], 'val': []}, 'mae': {'train': [], 'val': []}}

    for epoch in range(epochs):
        if model_type == 'hyp':
            eval_stats['loss']['train'].append(hyp_train_model(model, train_loader, criterion, optimizer, manifold, device))
            eval_stats['loss']['val'].append(util.h_evaluate_loss(model, val_loader, criterion, manifold, device))

            eval_stats['mae']['train'].append(util.h_evaluate_r2(model, train_loader, manifold, device))
            eval_stats['mae']['val'].append(util.h_evaluate_r2(model, val_loader, manifold, device))
        elif model_type == 'euc':
            eval_stats['loss']['train'].append(euc_train_model(model, train_loader, criterion, optimizer, device))
            eval_stats['loss']['val'].append(util.evaluate_loss(model, val_loader, criterion, device))

            eval_stats['mae']['train'].append(util.evaluate_r2(model, train_loader, device))
            eval_stats['mae']['val'].append(util.evaluate_r2(model, val_loader, device))

    print(eval_stats['mae']['val'])

Fold 0
[0.12213091453452084, 0.1157001481513608, 0.11392419176270517, 0.11284872166569462, 0.11235204818697327, 0.11316860040285931, 0.11410305083931009, 0.11440463729762851, 0.11531641660826678, 0.11545186484785316]
Fold 1
[0.12096299774953018, 0.11402811528347077, 0.1115709905687423, 0.1118897793812911, 0.1114985346409828, 0.1117588181582547, 0.11141746955268837, 0.1142810874286106, 0.11433417468577468, 0.11649154963442707]
Fold 2
[0.12097428843572282, 0.11386955485352537, 0.11219742989391258, 0.11092534413860397, 0.11160005394126267, 0.1127382517547978, 0.11300108278119132, 0.11431947476444206, 0.11581559038693329, 0.11611106004460434]
Fold 3
[0.12102868091192692, 0.11407396279351424, 0.1130131419674709, 0.11212084167886255, 0.11195427368953009, 0.1131073585755835, 0.11292370143948416, 0.11325685402463381, 0.11433557809975682, 0.11559678845224834]
Fold 4
[0.12071365008458936, 0.1139570541445054, 0.11124207452797544, 0.11129954140646825, 0.11089288999148879, 0.11139676387325646, 0.11

In [ ]:
Fold 0
[0.09094981766612582, 0.09608602443769439, 0.10213084733073284, 0.10297098476158782, 0.1039252772204806, 0.10574258559121522, 0.10593447081943014, 0.1103528270397624, 0.10905810868205902, 0.11674655082317638]
Fold 1
[0.09124620536624219, 0.09588467278224783, 0.1109519321222205, 0.10871296762480671, 0.11322182908185724, 0.10604926039894355, 0.11641335699831064, 0.11669891168893905, 0.1163291967061507, 0.11563811270315874]
Fold 2
[0.08261279344412803, 0.0949209955128152, 0.10394174902203669, 0.11074634671414854, 0.10959598998285282, 0.10492545479477039, 0.10590842284461503, 0.10538532105083259, 0.10268829907521582, 0.10696282556494001]
Fold 3
[0.09143749197812866, 0.10686854162056632, 0.10184344301563328, 0.10983260881492879, 0.11453995365690633, 0.10643641196149398, 0.10807662045371369, 0.11264002132608508, 0.11569218070241385, 0.11172063540914148]
Fold 4
[0.08755047684265042, 0.09826624201788955, 0.10408250848514021, 0.10678387971099373, 0.10168236861206509, 0.09372892354517087, 0.10054121141766975, 0.11307510662137904, 0.10818904212215745, 0.11473416400049585]